In [102]:
import pandas as pd
import numpy as np

print 'hello world'
a = [1,2,3]
b = [2,3,4]
a = np.array(a)
b = np.array(b)
print a
print b
print a+b
a = pd.DataFrame(a)
print a.mean()

hello world
[1 2 3]
[2 3 4]
[3 5 7]
0    2.0
dtype: float64


In [103]:

seed = 1024
np.random.seed(seed)
path = "../input/"

In [104]:
###
# 原则上这是一个<u,i,r>三项的推荐问题
# 最经典最常被提到的应该是amazon在电商上的应用
# 如何不从协同过滤而是从有监督机器学习的角度去思考这个问题
###

In [105]:
'''
读取数据成为一个pandas dataframe
'''
train = pd.read_csv(path+'invited_info_train.txt',dtype={"expert_id":str,'question_id':str})
test = pd.read_csv(path+'validate_nolabel.txt',dtype={"expert_id":str,'question_id':str}).fillna(-1)
test.columns = ['question_id','expert_id','label']
features = ['question_id','expert_id']
len_train = train.shape[0]

train = pd.concat([train,test])

In [106]:
'''
查看数据的前10行
'''
train.head(10)

,question_id,expert_id,label
0,1c525c9d44135bdc40ddd4b8d63738d8,e6a2ecac7f90d426103de95ba7f6d2b0,0.0
1,43c525e69431f916efcf8d09d99602ac,64e1c3152b0ad8ab1a3b6bbda5d2bbe8,1.0
2,e1a1009e93ea09bdd981029b592d89b8,405c2c59eb73d115b5d73d0d0cfc9c94,0.0
3,60f19c960a9300aabaf95ea49a9fe4c2,11ccc1bae27f606616070eca12441ffb,0.0
4,cb8f85a79be9cd9aae3af4a302b865a6,628b7aa38efb4c1ce999fe5cdf46a03e,0.0
5,51f87230b588bde394167c5f5189570b,7476aa28bb582c0ff10172f7e5773dba,0.0
6,2c8114aa8218d3499b70e2d74e795380,bd8044d8f3f9c3cff6c9fa51b20c1c4d,0.0
7,6928456febe78441e4df6dfcbfabfe73,362a43a6750a835a33a3fbc19fc6a5db,0.0
8,e888f91bd5d4710599cf217ee69dbdd6,acfde6daece350084e3a77e98972f2fb,0.0
9,400d208277095f149bbbd2f5dc45b501,9de7878db83d8799c37df99a50129d58,0.0


In [107]:
'''
对数据做初步的分析
'''

print 'length of train',train.shape[0]

print 'unique expert train',train['expert_id'].nunique()
print 'unique question train',train['question_id'].nunique()
print 'ratio expert',train['expert_id'].nunique()/float(train.shape[0])
print 'ratio question',train['question_id'].nunique()/float(train.shape[0])

print 'question answered by per expert',train.groupby('expert_id')["question_id"].count().mean()
print 'expert answering per question',train.groupby('question_id')["expert_id"].count().mean()

# [0,1,0,0,0,0,0,0,0,0,0]

length of train 276218
unique expert train 27937
unique question train 7778
ratio expert 0.10114112766
ratio question 0.0281589179561
question answered by per expert 9.88717471454
expert answering per question 35.5127282078


In [108]:
from sklearn.preprocessing import LabelEncoder
#对expert_id编码
le_expert = LabelEncoder()
le_expert.fit(train['expert_id'])
train['expert_id'] = le_expert.transform(train['expert_id'])
train['expert_id']

0        25169
1        11085
2         7004
3         1873
4        10824
5        12748
6        20730
7         5916
8        18981
9        17309
10       12049
11        4149
12        8336
13       24451
14       18654
15       19960
16        3794
17        5946
18         251
19        8248
20        6246
21        3726
22        8923
23        7150
24       15040
25        5368
26       19129
27        9042
28        8510
29       21234
         ...  
30436    14425
30437    16516
30438    24069
30439    24567
30440     7287
30441    20006
30442     9116
30443     1546
30444    24080
30445    24476
30446    26023
30447    17891
30448     7849
30449    13524
30450     3556
30451     6928
30452    14756
30453    24861
30454    24445
30455     7247
30456     1937
30457     2427
30458     6383
30459    11852
30460     6779
30461     8803
30462    23280
30463     3063
30464    27850
30465     1047
Name: expert_id, dtype: int64

In [109]:
#对question_id编码
le_question = LabelEncoder()
le_question.fit(train['question_id'])
train['question_id'] = le_question.transform(train['question_id'])
train['question_id'] 


0         891
1        2079
2        6861
3        2929
4        6209
5        2503
6        1402
7        3158
8        7056
9        1983
10       3341
11       5843
12       2341
13       3043
14       5324
15       3317
16       1194
17        176
18       2860
19       4319
20       7510
21       3030
22       2912
23       5005
24       7430
25       6144
26       6461
27       2031
28       1370
29       4560
         ... 
30436    4917
30437    6072
30438    5889
30439    6488
30440     745
30441    1889
30442    2548
30443    5202
30444    5617
30445    4174
30446    3158
30447    2350
30448    2056
30449    7395
30450    6142
30451    5727
30452    6674
30453    7746
30454    3348
30455    2155
30456    7704
30457    1550
30458    1702
30459    4473
30460    6826
30461    7361
30462    4473
30463    4917
30464    2395
30465    2957
Name: question_id, dtype: int64

In [110]:
train.head(10)
# '''
# 但是，即使转化为数字表示后，上述数据也不能直接用在我们的分类器中。
# 因为，线性分类器等往往默认数据数据是连续的，并且是有序的。但是，按照我们上述的表示，数字并不是有序的，而是随机分配的。
# '''

,question_id,expert_id,label
0,891,25169,0.0
1,2079,11085,1.0
2,6861,7004,0.0
3,2929,1873,0.0
4,6209,10824,0.0
5,2503,12748,0.0
6,1402,20730,0.0
7,3158,5916,0.0
8,7056,18981,0.0
9,1983,17309,0.0


In [111]:
# from sklearn.preprocessing import  OneHotEncoder
# #独热编码所有id特征
# enc = OneHotEncoder()
# enc.fit(train[features])
# X = enc.transform(train[features])
# y = train['label'].values
# X = X[:len_train]




In [112]:
# enc = OneHotEncoder()
# enc.fit(train[['expert_id']])
# X = enc.transform(train[['expert_id']])
# # y = train['label'].values
# # X = X[:len_train]
# # print X[:10].toarray()
# # 27937
# # 0,1
# print train['expert_id'].values[0]
# print X[0].toarray().ravel()[25169]

In [127]:
# print X[:10].toarray()
# print X.shape

X = train[features].values
'''
独热编码过后，得到的矩阵是一个稀疏矩阵，每一行代表一条记录
每一行中的元素只能为0或者1
'''

'\n\xe7\x8b\xac\xe7\x83\xad\xe7\xbc\x96\xe7\xa0\x81\xe8\xbf\x87\xe5\x90\x8e\xef\xbc\x8c\xe5\xbe\x97\xe5\x88\xb0\xe7\x9a\x84\xe7\x9f\xa9\xe9\x98\xb5\xe6\x98\xaf\xe4\xb8\x80\xe4\xb8\xaa\xe7\xa8\x80\xe7\x96\x8f\xe7\x9f\xa9\xe9\x98\xb5\xef\xbc\x8c\xe6\xaf\x8f\xe4\xb8\x80\xe8\xa1\x8c\xe4\xbb\xa3\xe8\xa1\xa8\xe4\xb8\x80\xe6\x9d\xa1\xe8\xae\xb0\xe5\xbd\x95\n\xe6\xaf\x8f\xe4\xb8\x80\xe8\xa1\x8c\xe4\xb8\xad\xe7\x9a\x84\xe5\x85\x83\xe7\xb4\xa0\xe5\x8f\xaa\xe8\x83\xbd\xe4\xb8\xba0\xe6\x88\x96\xe8\x80\x851\n'

In [128]:
'''
切分数据集
'''
from sklearn.model_selection import KFold

kf = KFold(n_splits=10,shuffle=True,random_state=1024).split(X)
for ind_tr,ind_te in kf:
    X_train = X[ind_tr]
    X_test = X[ind_te]
    y_train = y[ind_tr]
    y_test = y[ind_te]
    break



In [129]:
'''
训练模型
'''
from sklearn.linear_model import Ridge,Lasso
lr = Ridge(alpha=0,random_state=1024)
lr.fit(X_train,y_train)




Ridge(alpha=0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=1024, solver='auto', tol=0.001)

In [131]:
y_pred = lr.predict(X_test)
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0

In [132]:
'''
数据的评估
'''
from sklearn.metrics import accuracy_score,roc_auc_score
score = accuracy_score(y_test,y_pred)
print 'accuracy_score',score
# score = roc_auc_score(y_test,y_pred)
# print 'roc_auc_score',score
print y_test[:20]
print y_pred[:20]

id_1 = 2000
id_2 = 2001
id_3 = 40000


accuracy_score 0.791217145753
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.
  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


In [125]:
ground_truth = np.zeros(100)

ground_truth[:10]+=1

In [119]:
ground_truth

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [120]:
y_pred = np.zeros(100)
score = accuracy_score(ground_truth,y_pred)
print 'accuracy_score',score

accuracy_score 0.9
